In [8]:
import torch
import numpy as np
from tqdm import tqdm
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder

import os, sys
sys.path.append(os.path.abspath(os.path.join(os.getcwd(),'..')))
from utils import load_tensor, save_model
from models import BiLSTM, CNN_BiLSTM
from evaluation_functions import class_accuracy, class_f1_score

In [9]:
def remap_targets(target_tensor: torch.TensorType, old_le, new_le):
    inverse_tensor = old_le.inverse_transform(target_tensor.long())
    for idx, label in enumerate(inverse_tensor):
        if label == 'ARG_RESPONDENT' or label == 'ARG_PETITIONER':
            inverse_tensor[idx] = "ARG"
        elif label == 'PRE_NOT_RELIED' or label == 'PRE_RELIED':
            inverse_tensor[idx] = 'PRE'
    new_tensor = torch.tensor(new_le.transform(inverse_tensor))
    return new_tensor

In [10]:
list_of_targets_old = ['ISSUE', 'FAC', 'NONE', 'ARG_PETITIONER', 'PRE_NOT_RELIED', 'STA', 'RPC', 'ARG_RESPONDENT', 'PREAMBLE', 'ANALYSIS', 'RLC', 'PRE_RELIED', 'RATIO']
label_encoder_old = LabelEncoder().fit(list_of_targets_old)
list_of_targets_new = ['ISSUE', 'FAC', 'NONE', 'ARG', 'PRE', 'STA', 'RPC', 'PREAMBLE', 'ANALYSIS', 'RLC', 'RATIO']
label_encoder_new = LabelEncoder().fit(list_of_targets_new)

In [11]:
sample_input, sample_target = None, None
for idx in range(246):
    if sample_input is None:
        sample_input = load_tensor(filepath=f"../train_document/doc_{idx}/embedding")
        sample_target = load_tensor(filepath=f"../train_document/doc_{idx}/label")
    else:
        sample_input = torch.cat((sample_input,load_tensor(filepath=f"../train_document/doc_{idx}/embedding")), dim=0)
        sample_target = torch.cat((sample_target,load_tensor(filepath=f"../train_document/doc_{idx}/label")), dim=0)

In [12]:
sample_target.size()

torch.Size([28864])

In [13]:
remapped_target = remap_targets(sample_target, label_encoder_old, label_encoder_new)

In [14]:
from sklearn.utils.class_weight import compute_class_weight
    
class_weights = compute_class_weight(class_weight = "balanced",
                                    classes = np.unique(remapped_target.numpy()),
                                    y = remapped_target.numpy())
class_weights = torch.FloatTensor(class_weights)
class_weights

tensor([0.2460, 1.3107, 0.4584, 7.1499, 1.8544, 1.6639, 0.6347, 3.8990, 3.5033,
        2.4341, 5.4895])

In [17]:
def calculate_confusion_matrix(test_emb, test_labels, model, num_labels = 11):
    """

    Parameters:

    Returns:
    """
    model.eval()
    output = model(test_emb)
    return confusion_matrix(output, test_labels, num_labels)

def confusion_matrix(y_pred, y_true, num_classes):
    """
    Create a confusion matrix for label encodings in PyTorch.

    Parameters:
    y_pred (torch.Tensor): Predicted labels tensor.
    y_true (torch.Tensor): True labels tensor.
    num_classes (int): Number of classes.

    Returns:
    numpy.ndarray: Confusion matrix.
    """ 
    conf_matrix = np.zeros((num_classes, num_classes), dtype=np.int64)

    y_pred_np = y_pred.argmax(dim=1).cpu().numpy()
    y_true_np = y_true.cpu().numpy()

    for pred, true in zip(y_pred_np, y_true_np):
        conf_matrix[pred, true] += 1

    return conf_matrix

# BERT-Base

## BiLSTM

In [104]:
model1 = BiLSTM(hidden_size=128, dropout= 0.30, output_size= 11)
optimizer = torch.optim.Adam(model1.parameters(), lr= 2e-4)
loss_function = nn.CrossEntropyLoss(weight= class_weights)

print(f'{"Starting Training":-^100}')
model1.train()
loss_list = []
for epoch in range(100):
    running_loss = 0
    for idx in tqdm(range(246)):
        TRAIN_emb = load_tensor(filepath=f"../train_document/doc_{idx}/embedding")
        TRAIN_labels = load_tensor(filepath=f"../train_document/doc_{idx}/label")
        TRAIN_labels = remap_targets(TRAIN_labels, label_encoder_old, label_encoder_new)
        if TRAIN_emb.size(0) == 0:
            continue
        output = model1(TRAIN_emb)
        loss = loss_function(output,TRAIN_labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    # scheduler.step()
    # scheduler1.step()
    # scheduler2.step()
    # running_lr.append(model_opt.state_dict()['param_groups'][0]['lr'])
    loss_list.append(running_loss/246)
    print(f"Epoch: {epoch+1} \t Loss: {running_loss/246:.5f}")
    if running_loss/246 < 0.03:
        break
# batch_loss.append(loss.item())

cm = None
for i in range(29):
    TEST_emb = load_tensor(filepath=f"../test_document/doc_{i}/embedding")
    TEST_labels = load_tensor(filepath=f"../test_document/doc_{i}/label")
    TEST_labels = remap_targets(TEST_labels, label_encoder_old, label_encoder_new)
    conf_matrix_helper = calculate_confusion_matrix(TEST_emb, TEST_labels, model1, num_labels= 11)
    if cm is None:
        cm = conf_matrix_helper
    else:
        cm = np.add(cm, conf_matrix_helper)
        
accuracies = class_accuracy(cm)
f1_scores = class_f1_score(cm)
average_accuracy = np.mean(accuracies)
average_f1 = np.mean(f1_scores)

print("Accuracies: {} \n Average acccuracy: {}".format(accuracies, average_accuracy))
print("F1 Scores: {} \n Average F1: {}".format(f1_scores, average_f1))
# result.append((config, (average_accuracy, average_f1)))

-----------------------------------------Starting Training------------------------------------------


 76%|███████▌  | 186/246 [00:05<00:01, 35.37it/s]


KeyboardInterrupt: 

RUN 1

Accuracies: [0.82905028 0.40425532 0.77134146 0.78723404 0.94886364 0.53488372
 0.99401198 0.51470588 0.63076923 0.79207921 0.43902439] 

Average acccuracy: 0.6951108317340918

F1 Scores: [0.79358284 0.38383833 0.81943315 0.76288655 0.91256826 0.62670295
 0.99104473 0.49999995 0.45303863 0.8465608  0.51428566] 

Average F1: 0.6912674393701633

RUN 2

Accuracies: [0.81752701 0.40449438 0.7826087  0.73584906 0.96089385 0.44256757
 0.99013807 0.53521127 0.60714286 0.87234043 0.51351351] 

Average acccuracy: 0.6965715180230886

F1 Scores: [0.75331853 0.37305694 0.82420273 0.7572815  0.93224927 0.58482138
 0.99307611 0.53146848 0.39534879 0.90109885 0.57575752] 

Average F1: 0.6928800115505303

RUN 3

Accuracies: [0.84358974 0.39361702 0.75362319 0.74545455 0.93513514 0.42598187
 0.99404762 0.56896552 0.54054054 0.86021505 0.5       ] 

Average acccuracy: 0.6873791125060527

F1 Scores: [0.7498575  0.37373732 0.8195429  0.78095233 0.92266662 0.58385089
 0.99404757 0.50769226 0.26143787 0.88397785 0.52459011] 

Average F1: 0.6729412013793659

In [106]:
avg_acc = (0.6951108317340918 + 0.6965715180230886 + 0.6873791125060527) / 3
avg_f1 = (0.6912674393701633 + 0.6928800115505303 + 0.6729412013793659) / 3
print(f"{avg_acc:.4f}")
print(f"{avg_f1:.4f}")

0.6930
0.6857


## CNN-BiLSTM

In [110]:
model2 = CNN_BiLSTM(hidden_size=128, dropout= 0.30, output_size= 11)
optimizer = torch.optim.Adam(model2.parameters(), lr= 5e-4)
loss_function = nn.CrossEntropyLoss(weight= class_weights)

print(f'{"Starting Training":-^100}')
model2.train()
loss_list = []
for epoch in range(100):
    running_loss = 0
    for idx in tqdm(range(246)):
        TRAIN_emb = load_tensor(filepath=f"../train_document/doc_{idx}/embedding")
        TRAIN_labels = load_tensor(filepath=f"../train_document/doc_{idx}/label")
        TRAIN_labels = remap_targets(TRAIN_labels, label_encoder_old, label_encoder_new)
        if TRAIN_emb.size(0) == 0:
            continue
        output = model2(TRAIN_emb)
        loss = loss_function(output,TRAIN_labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    # scheduler.step()
    # scheduler1.step()
    # scheduler2.step()
    # running_lr.append(model_opt.state_dict()['param_groups'][0]['lr'])
    loss_list.append(running_loss/246)
    print(f"Epoch: {epoch+1} \t Loss: {running_loss/246:.5f}")
    if running_loss/246 < 0.03:
        break
# batch_loss.append(loss.item())
cm = None
for i in range(29):
    TEST_emb = load_tensor(filepath=f"../test_document/doc_{i}/embedding")
    TEST_labels = load_tensor(filepath=f"../test_document/doc_{i}/label")
    TEST_labels = remap_targets(TEST_labels, label_encoder_old, label_encoder_new)
    conf_matrix_helper = calculate_confusion_matrix(TEST_emb, TEST_labels, model2, num_labels= 11)
    if cm is None:
        cm = conf_matrix_helper
    else:
        cm = np.add(cm, conf_matrix_helper)
        
accuracies = class_accuracy(cm)
f1_scores = class_f1_score(cm)
average_accuracy = np.mean(accuracies)
average_f1 = np.mean(f1_scores)

print("Accuracies: {} \n Average acccuracy: {}".format(accuracies, average_accuracy))
print("F1 Scores: {} \n Average F1: {}".format(f1_scores, average_f1))

-----------------------------------------Starting Training------------------------------------------


  1%|          | 3/246 [00:00<00:10, 24.06it/s]

100%|██████████| 246/246 [00:07<00:00, 33.51it/s]


Epoch: 1 	 Loss: 1.60613


100%|██████████| 246/246 [00:07<00:00, 34.61it/s]


...

Epoch: 74 	 Loss: 0.03416


100%|██████████| 246/246 [00:08<00:00, 30.53it/s]


Epoch: 75 	 Loss: 0.03076


100%|██████████| 246/246 [00:07<00:00, 31.81it/s]


Epoch: 76 	 Loss: 0.02964
Accuracies: [0.83219955 0.61016949 0.75426136 0.77083333 0.88829787 0.57073171
 0.998      0.54054054 0.60377358 0.8        0.45652174] 
 Average acccuracy: 0.7113935617467742
F1 Scores: [0.7905223  0.44171774 0.82774742 0.75510199 0.88359783 0.65546214
 0.99402385 0.54794515 0.37869818 0.85106378 0.55999995] 
 Average F1: 0.6987163943896938


RUN 1

Accuracies: [0.82635342 0.59322034 0.78768233 0.77083333 0.92972973 0.65853659
 0.99405941 0.61842105 0.56338028 0.89010989 0.390625  ] 

Average acccuracy: 0.729359215774254

F1 Scores: [0.82804499 0.42944781 0.81270898 0.75510199 0.91733328 0.68354425
 0.99504455 0.63513508 0.42780744 0.90502788 0.53763436] 
 
Average F1: 0.7206209650612753

RUN 2

Accuracies: [0.82359192 0.53731343 0.77828746 0.7755102  0.94444444 0.66459627
 0.99013807 0.51807229 0.54794521 0.88888889 0.37037037] 

Average acccuracy: 0.7126507782608781

F1 Scores: [0.80897699 0.42105258 0.8256285  0.76767672 0.91891887 0.68370602
 0.99307611 0.55483866 0.42328038 0.89887635 0.48192766] 

Average F1: 0.7070871669091839

RUN 3

Accuracies: [0.83219955 0.61016949 0.75426136 0.77083333 0.88829787 0.57073171
 0.998      0.54054054 0.60377358 0.8        0.45652174] 

Average acccuracy: 0.7113935617467742

F1 Scores: [0.7905223  0.44171774 0.82774742 0.75510199 0.88359783 0.65546214
 0.99402385 0.54794515 0.37869818 0.85106378 0.55999995] 

Average F1: 0.6987163943896938


In [112]:
avg_acc = (0.729359215774254 + 0.7126507782608781 + 0.7113935617467742) / 3
avg_f1 = (0.7206209650612753 + 0.7070871669091839 + 0.6987163943896938) / 3
print(f"{avg_acc:.4f}")
print(f"{avg_f1:.4f}")

0.7178
0.7088


# LEGAL-BERT

## BiLSTM

In [114]:
while_i = 0
accs3 = []
f1s3 = []
micro_accs = []
micro_f1s = []
while while_i < 3:
    while_i += 1
    model3 = BiLSTM(hidden_size=128, dropout= 0.30, output_size= 11)
    optimizer = torch.optim.Adam(model3.parameters(), lr= 5e-4)
    loss_function = nn.CrossEntropyLoss(weight= class_weights)

    print(f'{"Starting Training":-^100}')
    model3.train()
    loss_list = []
    for epoch in range(100):
        running_loss = 0
        for idx in tqdm(range(246)):
            TRAIN_emb = load_tensor(filepath=f"../train_document/doc_{idx}_legal/embedding")
            TRAIN_labels = load_tensor(filepath=f"../train_document/doc_{idx}_legal/label")
            TRAIN_labels = remap_targets(TRAIN_labels, label_encoder_old, label_encoder_new)
            if TRAIN_emb.size(0) == 0:
                continue
            output = model3(TRAIN_emb)
            loss = loss_function(output,TRAIN_labels)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        # scheduler.step()
        # scheduler1.step()
        # scheduler2.step()
        # running_lr.append(model_opt.state_dict()['param_groups'][0]['lr'])
        loss_list.append(running_loss/246)
        print(f"Epoch: {epoch+1} \t Loss: {running_loss/246:.5f}")
        if running_loss/246 < 0.1:
            break
    cm = None
    for i in range(29):
        TEST_emb = load_tensor(filepath=f"../test_document/doc_{i}_legal/embedding")
        TEST_labels = load_tensor(filepath=f"../test_document/doc_{i}_legal/label")
        TEST_labels = remap_targets(TEST_labels, label_encoder_old, label_encoder_new)
        conf_matrix_helper = calculate_confusion_matrix(TEST_emb, TEST_labels, model3, num_labels= 11)
        if cm is None:
            cm = conf_matrix_helper
        else:
            cm = np.add(cm, conf_matrix_helper)
            
    accuracies = class_accuracy(cm)
    f1_scores = class_f1_score(cm)
    average_accuracy = np.mean(accuracies)
    average_f1 = np.mean(f1_scores)
    
    accs3.append(average_accuracy)
    f1s3.append(average_f1)
    micro_accs.append(accuracies)
    micro_f1s.append(f1_scores)

    print("Accuracies: {} \n Average acccuracy: {}".format(accuracies, average_accuracy))
    print("F1 Scores: {} \n Average F1: {}".format(f1_scores, average_f1))

-----------------------------------------Starting Training------------------------------------------


  0%|          | 0/246 [00:00<?, ?it/s]

100%|██████████| 246/246 [00:07<00:00, 32.29it/s]


Epoch: 1 	 Loss: 2.25662


100%|██████████| 246/246 [00:07<00:00, 31.90it/s]


...

Epoch: 98 	 Loss: 0.22095


100%|██████████| 246/246 [00:05<00:00, 41.54it/s]


Epoch: 99 	 Loss: 0.21726


100%|██████████| 246/246 [00:05<00:00, 43.15it/s]


Epoch: 100 	 Loss: 0.20940
Accuracies: [0.71106557 0.08125    0.53672316 0.61538462 0.74242424 0.16739447
 0.84140969 0.3        0.4        0.48854962 0.1038961 ] 
 Average acccuracy: 0.4534634070883946
F1 Scores: [0.47436769 0.0984848  0.59052054 0.25396822 0.38281246 0.27413585
 0.79749473 0.27272722 0.09160303 0.58447484 0.15094336] 
 Average F1: 0.3610484311527964


In [116]:
print(f"{np.mean(accs3):.4f}")
print(f"{np.mean(f1s3):.4f}")

0.4404
0.3679


## CNN-BiLSTM

In [117]:
while_i = 0
accs4 = []
f1s4 = []
micro_accs1 = []
micro_f1s1 = []
while while_i < 3:
    while_i += 1
    model4 = CNN_BiLSTM(hidden_size=128, dropout= 0.25, output_size= 11)
    optimizer = torch.optim.Adam(model4.parameters(), lr= 5e-4)
    loss_function = nn.CrossEntropyLoss(weight= class_weights)

    print(f'{"Starting Training":-^100}')
    model4.train()
    loss_list = []
    for epoch in range(100):
        running_loss = 0
        for idx in tqdm(range(246)):
            TRAIN_emb = load_tensor(filepath=f"../train_document/doc_{idx}_legal/embedding")
            TRAIN_labels = load_tensor(filepath=f"../train_document/doc_{idx}_legal/label")
            TRAIN_labels = remap_targets(TRAIN_labels, label_encoder_old, label_encoder_new)
            if TRAIN_emb.size(0) == 0:
                continue
            output = model4(TRAIN_emb)
            loss = loss_function(output,TRAIN_labels)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        # scheduler.step()
        # scheduler1.step()
        # scheduler2.step()
        # running_lr.append(model_opt.state_dict()['param_groups'][0]['lr'])
        loss_list.append(running_loss/246)
        print(f"Epoch: {epoch+1} \t Loss: {running_loss/246:.5f}")
        if running_loss/246 < 0.1:
            break
    cm = None
    for i in range(29):
        TEST_emb = load_tensor(filepath=f"../test_document/doc_{i}_legal/embedding")
        TEST_labels = load_tensor(filepath=f"../test_document/doc_{i}_legal/label")
        TEST_labels = remap_targets(TEST_labels, label_encoder_old, label_encoder_new)
        conf_matrix_helper = calculate_confusion_matrix(TEST_emb, TEST_labels, model4, num_labels= 11)
        if cm is None:
            cm = conf_matrix_helper
        else:
            cm = np.add(cm, conf_matrix_helper)
            
    accuracies = class_accuracy(cm)
    f1_scores = class_f1_score(cm)
    average_accuracy = np.mean(accuracies)
    average_f1 = np.mean(f1_scores)
    
    accs4.append(average_accuracy)
    f1s4.append(average_f1)
    micro_accs1.append(accuracies)
    micro_f1s1.append(f1_scores)

    print("Accuracies: {} \n Average acccuracy: {}".format(accuracies, average_accuracy))
    print("F1 Scores: {} \n Average F1: {}".format(f1_scores, average_f1))

-----------------------------------------Starting Training------------------------------------------


  1%|          | 3/246 [00:00<00:10, 22.54it/s]

100%|██████████| 246/246 [00:12<00:00, 19.91it/s]


Epoch: 1 	 Loss: 2.22142


100%|██████████| 246/246 [00:08<00:00, 28.41it/s]


...

Epoch: 98 	 Loss: 0.13259


100%|██████████| 246/246 [00:10<00:00, 24.24it/s]


Epoch: 99 	 Loss: 0.11272


100%|██████████| 246/246 [00:09<00:00, 26.60it/s]


Epoch: 100 	 Loss: 0.10305
Accuracies: [0.60479042 0.01075269 0.56730769 0.34210526 0.60714286 0.09821429
 0.88356164 0.26890756 0.15517241 0.46456693 0.04      ] 
 Average acccuracy: 0.367501977767658
F1 Scores: [0.65951488 0.01015223 0.58852863 0.2954545  0.27642273 0.08333328
 0.821656   0.33507849 0.10344823 0.54883716 0.03703699] 
 Average F1: 0.34176937400193425


In [118]:
print(f"{np.mean(accs4):.4f}")
print(f"{np.mean(f1s4):.4f}")

0.3777
0.3611


In [27]:
torch.save(model1, 'bert-base-bilstm.pth')
torch.save(model2, 'bert-base-cnnbilstm.pth')
torch.save(model3, 'bert-legal-bilstm.pth')
torch.save(model4, 'bert-legal-cnnbilstm.pth')

# InLEGAL BERT

In [28]:
import os, sys
sys.path.append(os.path.abspath(os.path.join(os.getcwd(),'..')))

from Dataset_Reader import Dataset_Reader
from utils import read_json, data_to_embeddings, save_tensor, label_encode, organize_data
from utils import document_max_length, write_dictionary_to_json

from main import TRAIN_DATA_PATH, TEST_DATA_PATH

In [29]:
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("law-ai/InLegalBERT")
model = AutoModel.from_pretrained("law-ai/InLegalBERT")

tokenizer_config.json: 100%|██████████| 516/516 [00:00<00:00, 419kB/s]
vocab.txt: 100%|██████████| 222k/222k [00:00<00:00, 1.06MB/s]
config.json: 100%|██████████| 671/671 [00:00<00:00, 492kB/s]
pytorch_model.bin: 100%|██████████| 534M/534M [00:51<00:00, 10.5MB/s] 
/home/kshitij/.local/lib/python3.8/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [32]:
train_data = Dataset_Reader('../data/train.json')
test_data = Dataset_Reader('../data/dev.json')

print(f"Number of sentences in training data: {len(train_data.texts)}")
print(f"Number of sentences in test data: {len(test_data.texts)}")

# Manually defining labels for convenience
list_of_targets = ['ISSUE', 'FAC', 'NONE', 'ARG_PETITIONER', 'PRE_NOT_RELIED',
                'STA', 'RPC', 'ARG_RESPONDENT', 'PREAMBLE', 'ANALYSIS', 'RLC', 'PRE_RELIED', 'RATIO']

# Numerically encode labels
label_encoder = label_encode(list_of_targets)


#Compute the maximum sentence length for each document in the training and test data (to ensure all embeddings will be the same size within a document)
max_length_dict_TRAIN = document_max_length(train_data, tokenizer=tokenizer)
max_length_dict_TEST = document_max_length(test_data, tokenizer=tokenizer)

# # To same time during training process, write these documents to json file
# write_dictionary_to_json(max_length_dict_TRAIN, 'max_length_dicts/max_length_train.json')
# write_dictionary_to_json(max_length_dict_TEST, 'max_length_dicts/max_length_test.json')


#retrieve max_length dictionaries to compute word embeddings
# max_length_dict_TRAIN = read_json('max_length_dicts/max_length_train.json', reading_max_length=True)
# max_length_dict_TEST = read_json('max_length_dicts/max_length_test.json', reading_max_length=True)


#organize and process data

train_doc_idxs, train_batched_texts, train_batched_labels = organize_data(train_data, batch_size= 1) 
test_doc_idxs, test_batched_texts, test_batched_labels = organize_data(test_data, batch_size= 1) 

for idx, train_idx in tqdm(enumerate(train_doc_idxs)):
    TRAIN_emb, TRAIN_labels = data_to_embeddings(train_idx, train_batched_texts[idx], train_batched_labels[idx],
                                                label_encoder,max_length_dict_TRAIN, tokenizer=tokenizer,
                                                emb_model=model)
    save_tensor(TRAIN_emb, 'train_document/doc_'+str(idx)+'_inlegal/',"embedding")
    save_tensor(TRAIN_labels, 'train_document/doc_'+str(idx)+'_inlegal/',"label")
    

for idx, test_idx in tqdm(enumerate(test_doc_idxs)):
    TEST_emb, TEST_labels = data_to_embeddings(test_idx, test_batched_texts[idx], test_batched_labels[idx],
                                            label_encoder,max_length_dict_TEST, tokenizer=tokenizer,
                                            emb_model=model)
    save_tensor(TEST_emb, 'test_document/doc_'+str(idx)+'_inlegal/',"embedding")
    save_tensor(TEST_labels, 'test_document/doc_'+str(idx)+'_inlegal/',"label")
    

Number of sentences in training data: 28986
Number of sentences in test data: 2890


1it [00:32, 32.08s/it]

X_train size: torch.Size([91, 1, 768])	Y_train size: torch.Size([91])
Tensor saved to 'train_document/doc_0_inlegal/embedding'
Tensor saved to 'train_document/doc_0_inlegal/label'


2it [01:07, 33.86s/it]

X_train size: torch.Size([72, 1, 768])	Y_train size: torch.Size([72])
Tensor saved to 'train_document/doc_1_inlegal/embedding'
Tensor saved to 'train_document/doc_1_inlegal/label'


...

X_train size: torch.Size([54, 1, 768])	Y_train size: torch.Size([54])
Tensor saved to 'test_document/doc_26_inlegal/embedding'
Tensor saved to 'test_document/doc_26_inlegal/label'


28it [20:09, 30.42s/it]

X_train size: torch.Size([43, 1, 768])	Y_train size: torch.Size([43])
Tensor saved to 'test_document/doc_27_inlegal/embedding'
Tensor saved to 'test_document/doc_27_inlegal/label'


29it [20:26, 42.30s/it]

X_train size: torch.Size([59, 1, 768])	Y_train size: torch.Size([59])
Tensor saved to 'test_document/doc_28_inlegal/embedding'
Tensor saved to 'test_document/doc_28_inlegal/label'


## BiLSTM

In [126]:
while_i = 0
accs5 = []
f1s5 = []
micro_accs2 = []
micro_f1s2 = []
while while_i < 3:
    while_i += 1
    model5 = BiLSTM(hidden_size=128, dropout= 0.3, output_size= 11)
    optimizer = torch.optim.Adam(model5.parameters(), lr= 5e-4)
    loss_function = nn.CrossEntropyLoss(weight= class_weights)

    print(f'{"Starting Training":-^100}')
    model5.train()
    loss_list = []
    for epoch in range(100):
        running_loss = 0
        for idx in tqdm(range(246)):
            TRAIN_emb = load_tensor(filepath=f"train_document/doc_{idx}_inlegal/embedding")
            TRAIN_labels = load_tensor(filepath=f"train_document/doc_{idx}_inlegal/label")
            TRAIN_labels = remap_targets(TRAIN_labels, label_encoder_old, label_encoder_new)
            if TRAIN_emb.size(0) == 0:
                continue
            output = model5(TRAIN_emb)
            loss = loss_function(output,TRAIN_labels)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        # scheduler.step()
        # scheduler1.step()
        # scheduler2.step()
        # running_lr.append(model_opt.state_dict()['param_groups'][0]['lr'])
        loss_list.append(running_loss/246)
        print(f"Epoch: {epoch+1} \t Loss: {running_loss/246:.5f}")
        if running_loss/246 < 0.025:
            break
    cm = None
    for i in range(29):
        TEST_emb = load_tensor(filepath=f"test_document/doc_{i}_inlegal/embedding")
        TEST_labels = load_tensor(filepath=f"test_document/doc_{i}_inlegal/label")
        TEST_labels = remap_targets(TEST_labels, label_encoder_old, label_encoder_new)
        conf_matrix_helper = calculate_confusion_matrix(TEST_emb, TEST_labels, model5, num_labels= 11)
        if cm is None:
            cm = conf_matrix_helper
        else:
            cm = np.add(cm, conf_matrix_helper)
            
    accuracies = class_accuracy(cm)
    f1_scores = class_f1_score(cm)
    average_accuracy = np.mean(accuracies)
    average_f1 = np.mean(f1_scores)
    
    accs5.append(average_accuracy)
    f1s5.append(average_f1)
    micro_accs2.append(accuracies)
    micro_f1s2.append(f1_scores)

    print("Accuracies: {} \n Average acccuracy: {}".format(accuracies, average_accuracy))
    print("F1 Scores: {} \n Average F1: {}".format(f1_scores, average_f1))

-----------------------------------------Starting Training------------------------------------------


  1%|          | 3/246 [00:00<00:11, 21.93it/s]

100%|██████████| 246/246 [00:07<00:00, 31.68it/s]


Epoch: 1 	 Loss: 1.43463


100%|██████████| 246/246 [00:08<00:00, 30.32it/s]


...

Epoch: 36 	 Loss: 0.02844


100%|██████████| 246/246 [00:09<00:00, 26.45it/s]


Epoch: 37 	 Loss: 0.02529


100%|██████████| 246/246 [00:08<00:00, 29.31it/s]


Epoch: 38 	 Loss: 0.02477
Accuracies: [0.85866261 0.41447368 0.78517398 0.86666667 0.87894737 0.32949309
 0.99015748 0.55769231 0.71428571 0.85393258 0.45238095] 
 Average acccuracy: 0.700169676236471
F1 Scores: [0.69197791 0.49218745 0.83709672 0.82105258 0.87894732 0.48805457
 0.9940711  0.46774189 0.27777775 0.85875701 0.53521122] 
 Average F1: 0.6675341369181542


In [127]:
print(f"{np.mean(accs5):.4f}")
print(f"{np.mean(f1s5):.4f}")

0.7031
0.6785


## CNN-BiLSTM

In [125]:
while_i = 0
accs6 = []
f1s6 = []
micro_accs3 = []
micro_f1s3 = []
while while_i < 3:
    while_i += 1
    model6 = BiLSTM(hidden_size=128, dropout= 0.3, output_size= 11)
    optimizer = torch.optim.Adam(model6.parameters(), lr= 5e-4)
    loss_function = nn.CrossEntropyLoss(weight= class_weights)

    print(f'{"Starting Training":-^100}')
    model6.train()
    loss_list = []
    for epoch in range(100):
        running_loss = 0
        for idx in tqdm(range(246)):
            TRAIN_emb = load_tensor(filepath=f"train_document/doc_{idx}_inlegal/embedding")
            TRAIN_labels = load_tensor(filepath=f"train_document/doc_{idx}_inlegal/label")
            TRAIN_labels = remap_targets(TRAIN_labels, label_encoder_old, label_encoder_new)
            if TRAIN_emb.size(0) == 0:
                continue
            output = model6(TRAIN_emb)
            loss = loss_function(output,TRAIN_labels)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        # scheduler.step()
        # scheduler1.step()
        # scheduler2.step()
        # running_lr.append(model_opt.state_dict()['param_groups'][0]['lr'])
        loss_list.append(running_loss/246)
        print(f"Epoch: {epoch+1} \t Loss: {running_loss/246:.5f}")
        if running_loss/246 < 0.025:
            break
    cm = None
    for i in range(29):
        TEST_emb = load_tensor(filepath=f"test_document/doc_{i}_inlegal/embedding")
        TEST_labels = load_tensor(filepath=f"test_document/doc_{i}_inlegal/label")
        TEST_labels = remap_targets(TEST_labels, label_encoder_old, label_encoder_new)
        conf_matrix_helper = calculate_confusion_matrix(TEST_emb, TEST_labels, model6, num_labels= 11)
        if cm is None:
            cm = conf_matrix_helper
        else:
            cm = np.add(cm, conf_matrix_helper)
            
    accuracies = class_accuracy(cm)
    f1_scores = class_f1_score(cm)
    average_accuracy = np.mean(accuracies)
    average_f1 = np.mean(f1_scores)

    accs6.append(average_accuracy)
    f1s6.append(average_f1)
    micro_accs3.append(accuracies)
    micro_f1s3.append(f1_scores)
    
    print("Accuracies: {} \n Average acccuracy: {}".format(accuracies, average_accuracy))
    print("F1 Scores: {} \n Average F1: {}".format(f1_scores, average_f1))

-----------------------------------------Starting Training------------------------------------------


  0%|          | 0/246 [00:00<?, ?it/s]

100%|██████████| 246/246 [00:20<00:00, 12.01it/s]


Epoch: 1 	 Loss: 1.44473


100%|██████████| 246/246 [00:12<00:00, 19.77it/s]


...

Epoch: 41 	 Loss: 0.06907


100%|██████████| 246/246 [00:09<00:00, 25.27it/s]


Epoch: 42 	 Loss: 0.03253


100%|██████████| 246/246 [00:09<00:00, 27.11it/s]


Epoch: 43 	 Loss: 0.02215
Accuracies: [0.84010152 0.39156627 0.75247525 0.69230769 0.89637306 0.5
 0.98821218 0.45588235 0.625      0.83157895 0.53658537] 
 Average acccuracy: 0.6827347846945421
F1 Scores: [0.75099258 0.48148143 0.82737165 0.7058823  0.90339421 0.58695647
 0.99308978 0.44285709 0.21428569 0.86338793 0.62857138] 
 Average F1: 0.6725700455676673


In [129]:
print(f"{np.mean(accs6):.4f}")
print(f"{np.mean(f1s6):.4f}")

0.6897
0.6810


## HOW DOES THE BILSTM WORK?

In [55]:
import torch.nn as nn

In [91]:
sample_input = load_tensor(filepath=f"train_document/doc_0_inlegal/embedding")

In [92]:
sample_input.size()

torch.Size([91, 1, 768])

In [93]:
# sample_input = sample_input.permute(1,2,0)
# sample_input.size()

In [94]:
bilstm = nn.LSTM(input_size=768, hidden_size=128, bidirectional= True)

In [95]:
output, (hidden, cell) = bilstm(sample_input)

In [96]:
output.size()

torch.Size([91, 1, 256])

In [97]:
hidden.size()

torch.Size([2, 1, 128])

In [98]:
cell.size()

torch.Size([2, 1, 128])